In [1]:
from pyomo.core import *
from pyomo.opt import SolverFactory, SolverManagerFactory
import pyomo.environ
import pandas as pd

from itertools import compress
import xlrd
import numpy as np
import time
start_time = time.time()

In [2]:
def Dict1D(dictVar,dataframe):
    for i,vali in enumerate(dataframe.index):
        dictVar[i+1]=dataframe.iloc[i][1]
        #dictVar[vali]=dataframe.iloc[i][1]
    return dictVar

def Dict1D_val_index(dictVar,dataframe):
    for i,vali in enumerate(dataframe.index):
        #dictVar[i+1]=dataframe.iloc[i][1]
        dictVar[vali]=dataframe.iloc[i][1]
    return dictVar

def DictND(dictVar,dataframe):
    for i,vali in enumerate(dataframe.index):
        for j,valj in enumerate(dataframe.columns):
            #dictVar[i+1,j+1]=dataframe.iloc[i][j+1]
            dictVar[vali,valj]=dataframe.iloc[i][j+1]
    return dictVar

In [199]:
excel_path=r'C:\Users\mobo\OneDrive\0PhD\Python-based energy hub\Generic_energy_hub_PYTHON\data\General_input.xlsx'
DemandData=pd.read_excel(excel_path,sheetname='Demand data')
Loads_init=DemandData.copy()
Loads_init.index=list(range(1,len(DemandData.index)+1))
Loads_init.columns=list(range(1,len(DemandData.columns)+1))
loads_init={}
loads_init=DictND(loads_init,Loads_init)

##################################
######## Solar data      #########
##################################

SolarData=pd.read_excel(excel_path,sheetname='Solar data')
SolarData.columns=[1]
solar_init={}
solar_init=Dict1D(solar_init,SolarData)


##################################
######## Technology parameters ###
##################################


Technologies=pd.read_excel(excel_path,sheetname='Technology', skiprows=1, index_col=0, skip_footer=38) #technology characteristics
Technologies=Technologies.dropna(axis=1, how='all') #technology characteristics
Technologies=Technologies.fillna(0) #technology characteristics

TechOutputs=pd.read_excel(excel_path,sheetname='Technology', skiprows=15, index_col=0, skip_footer=26) #Output matrix
TechOutputs=TechOutputs.dropna(axis=0,how='all')  #Output matrix
TechOutputs=TechOutputs.dropna(axis=1,how='all')  #Output matrix

Cost=TechOutputs.copy() #for costs

Dispatch_tech=pd.DataFrame(Cost.sum(0)) #find dispatch tech (CHP)
CHP_list=[]
for n,val in enumerate(Dispatch_tech[0]):
    if val>1:
        CHP_list.append(n+2) #first is electricity +1 since it starts at 0
        
        
Roof_tech=[]
for n,val in enumerate(Technologies.loc["Area"]):
    if val>0:
        Roof_tech.append(n+2) #first is electricity +1 since it starts at 0

#Based on the + - values, prepare data for generating coupling matrix
TechOutputs=TechOutputs.multiply(np.array(Technologies.loc['Efficiency']))
#TechOutputs.loc['Heat']=TechOutputs.loc['Heat'].multiply(np.array(Technologies.loc['HER'].fillna(value=1)))
TechOutputs.loc[TechOutputs.index!='Electricity']=TechOutputs.loc[TechOutputs.index!='Electricity'].multiply(np.array(Technologies.loc['HER'].fillna(value=1).replace(0,1)))
TechOutputs[TechOutputs<0]=-1

addGrid=np.zeros(len(DemandData.columns),)
addGrid[0]=1 #add electricity to coupling matrix
Grid=pd.DataFrame(addGrid,columns=["Grid"],index=DemandData.columns).transpose()

##################################
######## Coupling matrix #########
##################################

addGrid=np.zeros(len(DemandData.columns),)
addGrid[0]=1
Grid=pd.DataFrame(addGrid,columns=["Grid"],index=DemandData.columns).transpose()

Cmatrix=TechOutputs.transpose()

Cmatrix=pd.concat([Grid,Cmatrix])
Cmatrix.index=list(range(1,len(TechOutputs.columns)+2))
Cmatrix.columns=list(range(1,len(TechOutputs.index)+1))
cMatrix={}
cMatrix=DictND(cMatrix,Cmatrix)

##################################
######## PART LOAD  ##############
##################################
PartLoad=Technologies.loc["MinLoad (%)",]/100

partload=Cost.iloc[0:1].mul(list(PartLoad),axis=1)
partload=pd.concat([partload,Cost.iloc[1:].mul(list(PartLoad),axis=1)], axis=0)
partload=partload.abs()
partload=partload.transpose()
partload.index=list(range(1+1,len(TechOutputs.columns)+2))
partload.columns=list(range(1,len(TechOutputs.index)+1))
SolartechsSets=list(compress(list(range(1+1,len(Technologies.columns)+2)), list(Technologies.loc["Area"]>0)))

for i in SolartechsSets:
    partload.drop(i, inplace=True)
    
    
PartloadInput={}

PartloadInput=DictND(PartloadInput,partload)

##################################
########  Max capacity  ##########
##################################
MaxCap=Technologies.loc["Maximum Capacity",]
MaxCap.index=list(range(2,len(Technologies.loc["MinLoad (%)",].index)+2))
maxCap= MaxCap.to_dict()

for i in SolartechsSets:
    maxCap.pop(i, None)

##################################
########  Linear cost  ##########
##################################
LinearCost=Technologies.loc["CapCost (chf/kW)",]

linCost=Cost.iloc[0:1].mul(list(LinearCost),axis=1)
linCost=pd.concat([linCost,Cost.iloc[1:].mul(list(LinearCost),axis=1)], axis=0)

linCost=linCost.transpose()

for name in linCost.columns[1:]:
    linCost.loc[linCost["Electricity"] >1, name] = 0

linCost.loc[linCost["Electricity"] <0, "Electricity"]=0

linCost=linCost.abs()
linCost=pd.concat([Grid,linCost])

linCost.index=list(range(1,len(TechOutputs.columns)+2))
linCost.columns=list(range(1,len(TechOutputs.index)+1))
linCost.loc[1]=0

linCapCosts={}

linCapCosts=DictND(linCapCosts,linCost)


##############################################
########  LifeTime  And Interest Rate  #######
##############################################

Interest_rate=pd.read_excel(excel_path,sheetname='General', skiprows=8, index_col=0, skip_footer=7) #
Interest_rate=Interest_rate.dropna(axis=1,how='all')
Interest_rate_R=Interest_rate.loc["Interest Rate r"][0]

Life=pd.DataFrame(list(Technologies.loc["Lifetime (yr)"]))
Life.columns=[1]
Life.index=list(range(2,len(TechOutputs.columns)+2))
lifeTimeTechs={}
lifeTimeTechs=Dict1D_val_index(lifeTimeTechs, Life)

NetPresentValue=1 / (((1 + Interest_rate_R) ** Life - 1) / (Interest_rate_R * ((1 + Interest_rate_R) ** Life)))
NetPresentValueTech={}
NetPresentValueTech=Dict1D_val_index(NetPresentValueTech,NetPresentValue)



##########################################
########  Operational Maintence costs ####
##########################################

VarOMF=pd.DataFrame(list(Technologies.loc["OMVCost (chf/kWh)"]))
VarOMF.columns=[1]
VarOMF.index=list(range(1+1,len(TechOutputs.columns)+2))
VarOMF.loc[1]=0
omvCosts={}
omvCosts=Dict1D_val_index(omvCosts, VarOMF)

##################################
########  Carbon factors #########
##################################

Carbon=pd.read_excel(excel_path,sheetname='Technology', skiprows=24, index_col=0, skip_footer=16) #
Carbon=Carbon.dropna(axis=0,how='all')
Carbon=Carbon.dropna(axis=1,how='all')
Carbon.index=[1]

ElectricityCF=pd.read_excel(excel_path,sheetname='General', skiprows=1, index_col=0, skip_footer=14) #
ElectricityCF=ElectricityCF.dropna(axis=0,how='all')
ElectricityCF=ElectricityCF.dropna(axis=1,how='all')
del ElectricityCF["Price (chf/kWh)"]
ElectricityCF.index=[1]

CarbonFactors= pd.concat([ElectricityCF, Carbon], axis=1)
CarbonFactors.columns=list(range(1,len(CarbonFactors.columns)+1))
CarbonFactors=CarbonFactors.transpose()

carbonFactors={}
carbonFactors=Dict1D(carbonFactors,CarbonFactors)



##############################################
########  Fuel data                ###########
##############################################

Fuel=pd.read_excel(excel_path,sheetname='General', skiprows=1, index_col=0, skip_footer=10) #
Fuel=Fuel.dropna(axis=0,how='all')

opPrice=CarbonFactors.copy()

for n,val in enumerate(Fuel["CO2 (kg/kWh)"]):
    for index,value in opPrice.iterrows():
        if float(val)==float(value):
            opPrice.loc[index]=float(Fuel["Price (chf/kWh)"][n])
            
opPrices={}
opPrices=Dict1D(opPrices,opPrice)


##############################################
########  Feedin tariff            ###########
##############################################

Tariff=pd.read_excel(excel_path,sheetname='General', skiprows=11, index_col=0, skip_footer=0) #
Tariff=Tariff.dropna(axis=0,how='all')
Tariff=Tariff.dropna(axis=1,how='all')
Tariff.columns=[1]
Tariff.index=list(range(1,len(DemandData.columns)+1))

feedInTariffs={}
feedInTariffs=Dict1D_val_index(feedInTariffs,Tariff)


##################################
########  Storage    #############
##################################

Storage=pd.read_excel(excel_path,sheetname='Technology', skiprows=40, index_col=0, skip_footer=0) #
Storage=Storage.dropna(axis=1, how='all')
Storage=Storage.fillna(0)

maxStorCh={}
MaxCharge=pd.DataFrame(list(Storage.loc["max_charge"]))
MaxCharge.columns=[1]
maxStorCh=Dict1D(maxStorCh, MaxCharge)

maxStorDisch={}
MaxDischarge=pd.DataFrame(list(Storage.loc["max_discharge"]))
MaxDischarge.columns=[1]
maxStorDisch=Dict1D(maxStorDisch, MaxDischarge)

lossesStorStanding={}
losses=pd.DataFrame(list(Storage.loc["decay"]))
losses.columns=[1]
lossesStorStanding=Dict1D(lossesStorStanding, losses)

chargingEff={}
Ch_eff=pd.DataFrame(list(Storage.loc["ch_eff"]))
Ch_eff.columns=[1]
chargingEff=Dict1D(chargingEff, Ch_eff)

dischLosses={}
Disch_eff=pd.DataFrame(list(Storage.loc["disch_eff"]))
Disch_eff.columns=[1]
dischLosses=Dict1D(dischLosses, Disch_eff)

minSoC={}
min_state=pd.DataFrame(list(Storage.loc["min_state"]))
min_state.columns=[1]
minSoC=Dict1D(minSoC, min_state)

lifeTimeStorages={}
LifeBattery=pd.DataFrame(list(Storage.loc["LifeBat (year)"]))
LifeBattery.columns=[1]
lifeTimeStorages=Dict1D(lifeTimeStorages, LifeBattery)

linStorageCosts={}
LinearCostStorage=pd.DataFrame(list(Storage.loc["CostBat (chf/kWh)"]))
LinearCostStorage.columns=[1]
linStorageCosts=Dict1D(linStorageCosts, LinearCostStorage)

NetPresentValueStor={}
NetPresentValueStorage=1 / (((1 + Interest_rate_R) ** LifeBattery - 1) / (Interest_rate_R * ((1 + Interest_rate_R) ** LifeBattery)))
NetPresentValueStor=Dict1D(NetPresentValueStor,NetPresentValueStorage)

In [220]:
list(partload.loc[partload.sum(axis=1)>0].index)

[4, 7, 8]

In [94]:
model.In.data()

{1, 2, 3, 4, 5, 6, 7, 8, 9}

In [209]:
CHP_list

[4, 7, 8]

In [236]:
#-----------------------------------------------------------------------------#
## Creating a model ##
#-----------------------------------------------------------------------------#
#model = AbstractModel()
model = ConcreteModel()

# number of hours, technologies, timesteps
#model.hours = Param(within=NonNegativeIntegers)
#model.hours = Param(within=NonNegativeIntegers, initilize=DemandData.shape[0])

#model.technologies = Param(within=NonNegativeIntegers,initilize=Technologies.shape[0])
#model.demand = Param(within=NonNegativeIntegers)
# sets definition
model.Time = RangeSet(1, DemandData.shape[0])
model.SubTime = RangeSet(2, DemandData.shape[0], within=model.Time)
model.In = RangeSet(1, Technologies.shape[1]+1)
model.W_Ogrid = Set(initialize=list(range(1+1,len(Technologies.columns)+2)), within=model.In) #techs w/o grid


model.Out = RangeSet(1, DemandData.shape[1])
number_of_demands= list(range(1, DemandData.shape[1]+1))

#SolartechsSets=list(compress(list(range(1+1,len(Technologies.columns)+2)), list(Technologies.loc["Area"]>0)))
model.SolarTechs = Set(initialize=SolartechsSets, within=model.In)

DispTechsSet=list(compress(list(range(1+1,len(Technologies.columns)+2)), list(Technologies.loc["Area"]==0)))
partloadtechs=list(partload.loc[partload.sum(axis=1)>0].index)
model.DispTechs = Set(initialize=DispTechsSet, within=model.In)
model.Techs = Set(initialize=list(maxCap.keys()), within=model.In)
model.PartLoad=Set(initialize=partloadtechs, within=model.In)

model.CHP = Set(initialize=CHP_list, within=model.In) # set dispatch tech set
model.roof_tech = Set(initialize=Roof_tech, within=model.In) # set tech with roof area set
model.gas_tech = Set(initialize=[3,4,7,8], within=model.In) #

######################################################
### Find which is the primary input for capacity #####
######################################################

dispatch_demands=np.zeros((len(CHP_list), 2), dtype=int)

for n,val in enumerate(CHP_list):
    counter=0
    for i, value in enumerate(np.array(Cost[[val-2]],dtype=int)):
        if value[0]>0 and counter==0:
            dispatch_demands[n,0]=i+1
            counter=1
        if value[0]>0 and counter==1:
            dispatch_demands[n,1]=i+1
#model.CHP_in = Set(initialize=list(dispatch_demands[:,0]), within=model.Out)
#model.CHP_out = Set(initialize=list(dispatch_demands[:,1]), within=model.Out)

# coupling matrix & Technical parameters
model.cMatrix = Param(model.In, model.Out, initialize=cMatrix)                                      # coupling matrix technology efficiencies 
model.maxCapTechs = Param(model.DispTechs, initialize=maxCap)
model.maxCapTechsAll = Param(model.Techs, initialize=maxCap)
model.maxStorCh = Param(model.Out, initialize=maxStorCh)
model.maxStorDisch = Param(model.Out, initialize= maxStorDisch)

model.lossesStorStanding = Param(model.Out, initialize = lossesStorStanding)
model.chargingEff = Param(model.Out, initialize = chargingEff)
model.dischLosses = Param(model.Out, initialize = dischLosses)
model.minSoC = Param(model.Out, initialize = minSoC)
model.partLoad = Param(model.In, model.Out, initialize=PartloadInput) #PartloadInput
model.maxSolarArea = Param(initialize=50)

# carbon factors
model.carbonFactors = Param(model.In, initialize=carbonFactors)
model.maxCarbon = Param(initialize=3000000)

# Cost parameters
model.linCapCosts = Param(model.In, model.Out, initialize= linCapCosts)         # Technologies capital costs
#model.fixCapCosts = Param(model.In, model.Out)
model.linStorageCosts = Param(model.Out, initialize = linStorageCosts)
model.opPrices = Param(model.In, initialize=opPrices)                       # Operating prices technologies
model.feedInTariffs = Param(model.Out, initialize=feedInTariffs)               # feed-in tariffs
model.omvCosts = Param(model.In, initialize=omvCosts)                            # Maintenance costs
model.interestRate = Param(within=NonNegativeReals, initialize=Interest_rate_R)
# lifetime
model.lifeTimeTechs = Param(model.In, initialize = lifeTimeTechs)
model.lifeTimeStorages = Param(model.Out, initialize = lifeTimeStorages)


## Declaring Global Parameters ##
model.timeHorizon = Param(within=NonNegativeReals, initialize=20)
model.bigM = Param(within=NonNegativeReals, initialize=10000)

#loads

model.loads = Param(model.Time, model.Out, initialize=loads_init)
model.solarEm = Param(model.Time, initialize=solar_init)


## Global variables
model.P = Var(model.Time, model.In, domain=NonNegativeReals)
model.Pexport = Var(model.Time, model.Out, domain=NonNegativeReals)
model.Capacities = Var(model.In, model.Out, domain=NonNegativeReals)
model.Ytechnologies = Var(model.In, model.Out, domain=Binary)
model.Yon = Var(model.Time, model.In, domain=Binary)
model.TotalCost = Var(domain=Reals)
model.OpCost = Var(domain=NonNegativeReals)
model.MaintCost = Var(domain=NonNegativeReals)
model.IncomeExp = Var(domain=NonNegativeReals)
model.InvCost = Var(domain=NonNegativeReals)
model.TotalCarbon = Var(domain=Reals)
model.NetPresentValueTech = Param(model.In, domain=NonNegativeReals, initialize=NetPresentValueTech)
model.NetPresentValueStor = Param(model.Out, domain=NonNegativeReals, initialize=NetPresentValueStor)
#Storage variables
model.Qin = Var(model.Time, model.Out, domain=NonNegativeReals)
model.Qout = Var(model.Time, model.Out, domain=NonNegativeReals)
model.E = Var(model.Time, model.Out, domain=NonNegativeReals)
model.StorageCap = Var(model.Out)
model.Ystorage = Var(model.Out, domain=Binary)
#model.maxStorageCap = Param(model.Out, initialize= maxStorageCap)

In [32]:
dispatch_demands=np.zeros((len(CHP_list), 2), dtype=int)
for n,val in enumerate(CHP_list):
    counter=0
    for i, value in enumerate(np.array(Cost[[val-2]],dtype=int)):
        if value[0]>0 and counter==0:
            dispatch_demands[n,0]=i+1
            counter=1
        if value[0]>0 and counter==1:
            dispatch_demands[n,1]=i+1

In [44]:
d={}
for x in range(1,dispatch_demands.shape[0]+1):
        d["CHP{0}_D1".format(x)]=dispatch_demands[x-1,0]
        d["CHP{0}_D2".format(x)]=dispatch_demands[x-1,1]

In [237]:
#-----------------------------------------------------------------------------#
## GLobal constraints
#-----------------------------------------------------------------------------#
def loadsBalance_rule(model, t, out):
    return (model.loads[t,out] + model.Pexport[t,out] <= (model.Qout[ t,out] - model.Qin[t,out] + 
                                                        sum(model.P[t,inp]*model.cMatrix[inp,out] for inp in model.In)))
model.loadsBalance = Constraint(model.Time, model.Out, rule=loadsBalance_rule)

def capacityConst_rule(model, t, inp, out):
    if model.cMatrix[inp,out] <= 0:
        return(Constraint.Skip)
    else:
        return (model.P[t, inp]  * model.cMatrix[inp,out]<= model.Capacities[inp,out])
model.capacityConst = Constraint(model.Time, model.In, model.Out, rule=capacityConst_rule)

def maxCapacity_rule2(model, tech, out):
    return (model.Capacities[tech, out] <= model.maxCapTechs[tech])
model.maxCapacity2 = Constraint(model.DispTechs, model.Out, rule=maxCapacity_rule2)



def capacity_rule(model, inp, out):
    if model.cMatrix[inp,out] <= 0:
        return(model.Capacities[inp,out] == 0)
    else:
        return(Constraint.Skip)
model.capacity_feasibility = Constraint(model.In, model.Out, rule=capacity_rule)


#def maxCapacityBIS_rule(model):
#    return (model.Capacities[2, 2] == 10)
#model.maxCapacityBIS = Constraint(rule=maxCapacityBIS_rule)

def carbonConst_rule(model):
    return (model.TotalCarbon <= model.maxCarbon)
model.carbonConst = Constraint(rule=carbonConst_rule)

In [238]:
#-----------------------------------------------------------------------------#
## Specific constraints 
#-----------------------------------------------------------------------------#

def partLoadL_rule(model, t, disp, out):  
    if model.cMatrix[disp,out] <= 0:
        return(Constraint.Skip)
    else:
        return (model.partLoad[disp, out] * model.Capacities[disp, out] <= 
                                                                    (model.P[disp, out] * model.cMatrix[disp,out] 
                                                                    + model.bigM * (1 - model.Yon[t, disp])))
model.partLoadL = Constraint(model.Time, model.PartLoad, model.Out, rule=partLoadL_rule)

def partLoadU_rule(model, t, disp, out):    
    if model.cMatrix[disp,out] <= 0:
        return(Constraint.Skip)
    else:
        return (model.P[t, disp] * model.cMatrix[disp, out] <= model.bigM * model.Yon[t, disp])
model.partLoadU = Constraint(model.Time, model.PartLoad, model.Out, rule=partLoadU_rule)



def solarInput_rule(model, t, sol, out):
    if model.cMatrix[sol, out] <= 0:
        return(Constraint.Skip)
    else:
        return (model.P[t, sol] == model.solarEm[t] * model.Capacities[sol, out])
model.solarInput = Constraint(model.Time, model.SolarTechs, model.Out, rule=solarInput_rule) 


def roofArea_rule(model,roof, demand):
    return (sum(model.Capacities[roof,d] for d in number_of_demands) <= model.maxSolarArea) #DemandData.shape[1]
model.roofArea = Constraint(model.roof_tech,model.Out,rule=roofArea_rule) #model.roof_tech

def fixCostConst_rule(model, inp, out):
    return (model.Capacities[inp,out] <= model.bigM * model.Ytechnologies[inp,out])
model.fixCostConst = Constraint(model.In, model.Out, rule=fixCostConst_rule)


model.con = ConstraintList()
for x in range(1, len(dispatch_demands)+1):
    #globals()['model.h2pRatioTesting%s' % x] = Constraint(expr=model.Capacities[CHP_list[x-1],dispatch_demands[x-1,1]] == model.cMatrix[CHP_list[x-1],dispatch_demands[x-1,1]] / model.cMatrix[CHP_list[x-1],dispatch_demands[x-1,0]] * model.Capacities[CHP_list[x-1],dispatch_demands[x-1,0]])
    model.con.add(model.Capacities[CHP_list[x-1],dispatch_demands[x-1,1]] == model.cMatrix[CHP_list[x-1],dispatch_demands[x-1,1]] / model.cMatrix[CHP_list[x-1],dispatch_demands[x-1,0]] * model.Capacities[CHP_list[x-1],dispatch_demands[x-1,0]])
    model.con.add(model.Ytechnologies[CHP_list[x-1],dispatch_demands[x-1,0]]==model.Ytechnologies[CHP_list[x-1],dispatch_demands[x-1,1]])
    model.con.add(model.Capacities[CHP_list[x-1],dispatch_demands[x-1,0]] <= model.maxCapTechs[CHP_list[x-1]] * model.Ytechnologies[CHP_list[x-1],dispatch_demands[x-1,0]])

#model.con.construct()

In [172]:
for val in model.con.values():
    print (val.body)

Capacities[4,2] - 1.7300000000000002*Capacities[4,1]
Ytechnologies[4,1] - Ytechnologies[4,2]
Capacities[4,1] - 100.0*Ytechnologies[4,1]
Capacities[7,3] - 1.7300000000000002*Capacities[7,1]
Ytechnologies[7,1] - Ytechnologies[7,3]
Capacities[7,1] - 100.0*Ytechnologies[7,1]
Capacities[8,2] - 1.7300000000000002*Capacities[8,1]
Ytechnologies[8,1] - Ytechnologies[8,2]
Capacities[8,1] - 100.0*Ytechnologies[8,1]


In [104]:
for x in range(1, len(dispatch_demands)+1):
    globals()['model.h2pRatioTesting%s' % x] = Constraint(expr=model.Capacities[CHP_list[x-1],dispatch_demands[x-1,1]] == model.cMatrix[CHP_list[x-1],dispatch_demands[x-1,1]] / model.cMatrix[CHP_list[x-1],dispatch_demands[x-1,0]] * model.Capacities[CHP_list[x-1],dispatch_demands[x-1,0]])

In [239]:
#-----------------------------------------------------------------------------#
## Storage constraints
#-----------------------------------------------------------------------------#

def storageBalance_rule(model, t, out):
    return (model.E[t, out] == (
                                model.lossesStorStanding[out] * model.E[(t-1), out] 
                                + model.chargingEff[out] * model.Qin[t, out] 
                                - (1/model.dischLosses[out]) * model.Qout[t, out]))
model.storageBalance = Constraint(model.SubTime, model.Out, rule=storageBalance_rule)

model.StorCon = ConstraintList()
for x in range(1, DemandData.shape[1]+1):
    
    model.StorCon.add(model.E[1, x] == model.StorageCap[x] * model.minSoC[x])
    model.StorCon.add(model.E[1, x] == model.E[8760, x])
    model.StorCon.add(model.Qout[1, x] == 0)
    

'''
def storageInitBattery_rule(model):
    return (model.E[1, 1] == model.StorageCap[1] * model.minSoC[1])
model.storageInitBattery = Constraint(rule=storageInitBattery_rule)

def storageInitThermal1_rule(model):
    return (model.E[1, 2] == model.E[8760, 2])
model.storageInitThermal1 = Constraint(rule=storageInitThermal1_rule)

def storageInitThermal2_rule(model):
    return (model.Qout[1, 2] == 0)
model.storageInitThermal2 = Constraint(rule=storageInitThermal2_rule)
'''
def storageChargeRate_rule(model, t, out):
    return (model.Qin[t,out] <= model.maxStorCh[out] * model.StorageCap[out])
model.storageChargeRate = Constraint(model.Time, model.Out, rule=storageChargeRate_rule)

def storageDischRate_rule(model, t, out):
    return (model.Qout[t,out] <= model.maxStorDisch[out] * model.StorageCap[out])
model.storageDischRate = Constraint(model.Time, model.Out, rule=storageDischRate_rule)

def storageMinState_rule(model, t, out):
    return (model.E[t,out] >= model.StorageCap[out] * model.minSoC[out])
model.storageMinState = Constraint(model.Time, model.Out, rule=storageMinState_rule)

def storageCap_rule(model, t, out):
    return (model.E[t,out] <= model.StorageCap[out] )
model.storageCap = Constraint(model.Time, model.Out, rule=storageCap_rule)

#def storageMaxCap_rule(model, t, out):
#    return (model.E[t,out] <= model.maxStorageCap[out] )
#model.storageMaxCap = Constraint(model.Time, model.Out, rule=storageMaxCap_rule)


In [240]:
#-----------------------------------------------------------------------------#
## Objective functions
#-----------------------------------------------------------------------------#
def objective_rule(model):
    return (model.TotalCost)
model.Total_Cost = Objective(rule=objective_rule, sense=minimize)

#def objective_rule(model):
#    return (model.TotalCarbon)
#model.Total_Carbon = Objective(rule=objective_rule, sense=minimize)

def opCost_rule(model):
    return(model.OpCost == ((sum (model.opPrices[inp] 
                            * sum(model.P[t,inp] for t in model.Time)
                            for inp in model.In)))
                            )
model.opCost = Constraint(rule=opCost_rule) 

def maintenanceCost_rule(model):
    return(model.MaintCost == (sum(model.P[t,inp] * 
                              model.cMatrix[inp,out] * model.omvCosts[inp]
                              for t in model.Time for inp in model.In for out in model.Out)
                              ))
model.maintCost = Constraint(rule=maintenanceCost_rule)

def incomeExp_rule(model):
    return(model.IncomeExp == (sum(model.feedInTariffs[out] * 
                            sum(model.Pexport[t,out] for t in model.Time) 
                            for out in model.Out))
                            )
model.incomeExp = Constraint(rule=incomeExp_rule)

def invCost_rule(model):
    return(model.InvCost == (sum(model.NetPresentValueTech[inp] * (model.linCapCosts[inp,out] * model.Capacities[inp,out] ) for inp in model.W_Ogrid for out in model.Out) #+ model.fixCapCosts[inp,out] * model.Ytechnologies[inp,out]
                            + sum(model.NetPresentValueStor[out] * model.linStorageCosts[out] * model.StorageCap[out] for out in model.Out))
                            )
model.invCost = Constraint(rule=invCost_rule) 

def totalCost_rule(model):
    return(model.TotalCost == model.InvCost + model.OpCost + model.MaintCost - model.IncomeExp)
model.totalCost = Constraint(rule=totalCost_rule) 

def totalCarbon_rule(model):
    return(model.TotalCarbon == sum(model.carbonFactors[inp] * sum(model.P[t,inp] for t in model.Time) for inp in model.In))
model.totalCarbon = Constraint(rule=totalCarbon_rule)


In [241]:
#-----------------------------------------------------------------------------#
## Print model ##
#-----------------------------------------------------------------------------#
#model.pprint()

#instance = model.create(r'C:\Users\mobo\OneDrive\0PhD\Python-based energy hub\Generic_energy_hub_PYTHON\generic_model.dat')


opt = SolverFactory("gurobi")
opt.options["mipgap"]=0.05
opt.options["FeasibilityTol"]=1e-05
solver_manager = SolverManagerFactory("serial")
#results = solver_manager.solve(instance, opt=opt, tee=True,timelimit=None, mipgap=0.1)

results = solver_manager.solve(model, opt=opt, tee=True,timelimit=None)

#sends results to stdout
#results.write()


def pyomo_save_results(options=None, instance=None, results=None):
    OUTPUT = open(r'C:\Users\mobo\OneDrive\0PhD\Python-based energy hub\Generic_energy_hub_PYTHON\data\Results_generic_hub.txt','w')
    print(results, file=OUTPUT)
    OUTPUT.close()
#pyomo_save_results(instance=instance, results=results)    

#Example of how to print variables
#print(instance.TotalCost.value)
print(model.TotalCost.value)




#for i in instance.Pnaturalgas:
#    print (instance.Pnaturalgas[i], instance.Pnaturalgas[i].value)

#print("--- %s seconds ---" % (time.time() - start_time))

## create the instance
#instance = model.create(’DiseaseEstimation.dat’)
## define the solver and its options
#solver = ’ipopt’
#opt = SolverFactory( solver )
#if opt is None:
#raise ValueError, "Problem constructing solver \
#‘"+str(solver)
#opt.set_options(’max_iter=2’)
## create the solver manager
#solver_manager = SolverManagerFactory( ’serial’ )
## solve
#results = solver_manager.solve(instance, opt=opt, tee=True,\
#timelimit=None)
#instance.load(results)
## display results
#display(instance)



Changed value of parameter MIPGap to 0.05
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Parameter mipgap unchanged
   Value: 0.05  Min: 0.0  Max: 1e+100  Default: 0.0001
Changed value of parameter FeasibilityTol to 1e-05
   Prev: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Optimize a model with 385529 rows, 210304 columns and 1190843 nonzeros
Variable types: 183997 continuous, 26307 integer (26307 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+06]
Presolve removed 113993 rows and 43875 columns (presolve time = 24s) ...
Presolve removed 113993 rows and 43875 columns
Presolve time: 24.18s
Presolved: 271536 rows, 166429 columns, 805849 nonzeros
Variable types: 140149 continuous, 26280 integer (26280 binary)
Presolve removed 271536 rows and 166429 columns

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.2262600e+3

In [206]:
Cap=model.Capacities.get_values()

In [242]:
model.Capacities.get_values()

{(1, 1): 10000.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (2, 1): 0.0,
 (2, 2): 30.2029259084,
 (2, 3): 0.0,
 (3, 1): 0.0,
 (3, 2): 50.0,
 (3, 3): 0.0,
 (4, 1): 4.0,
 (4, 2): 6.92,
 (4, 3): 0.0,
 (5, 1): 0.0,
 (5, 2): 0.0,
 (5, 3): 0.0,
 (6, 1): 0.0,
 (6, 2): 0.0,
 (6, 3): 0.0,
 (7, 1): 23.1213872832,
 (7, 2): 0.0,
 (7, 3): 40.0,
 (8, 1): 5.66304860783,
 (8, 2): 9.79707409155,
 (8, 3): 0.0,
 (9, 1): 0.0,
 (9, 2): 0.0,
 (9, 3): 50.0}

In [243]:
model.StorageCap.get_values()

{1: 0.0, 2: 0.0, 3: 0.0}

In [189]:
P_matrix=np.zeros(shape=(DemandData.shape[0],Technologies.shape[1]+1))
for i in range(1,DemandData.shape[0]+1):
    for j in range(1,Technologies.shape[1]+1+1):
        P_matrix[i-1,j-1]=model.P[i,j].value

In [190]:
P=pd.DataFrame(P_matrix)

In [191]:
P

,0,1,2,3,4,5,6,7,8
0,0.000000,0.0,16.819149,10.000000,0.0,0.0,0.000000,0.0,22.340426
1,0.000000,0.0,20.010638,10.000000,0.0,0.0,0.000000,0.0,25.531915
2,0.000000,0.0,22.138298,10.000000,0.0,0.0,0.000000,0.0,27.659574
3,0.000000,0.0,24.265957,10.000000,0.0,0.0,0.000000,0.0,29.787234
4,0.000000,0.0,26.393617,10.000000,0.0,0.0,0.000000,0.0,31.914894
5,0.000000,0.0,27.457447,10.000000,0.0,0.0,0.000000,0.0,32.978723
6,0.000000,0.0,53.191489,44.315992,0.0,0.0,44.315992,0.0,53.191489
7,0.000000,0.0,44.904255,45.838150,0.0,0.0,30.828516,0.0,53.191489
8,0.000000,0.0,36.680851,54.951830,0.0,0.0,25.048170,0.0,53.191489
9,0.000000,0.0,35.244681,47.919075,0.0,0.0,15.414258,0.0,53.191489
